In [2]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict
import os

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [3]:
rr = RandomRules(
  TOTAL_NUMBERS = 35,
  CHOOSE = 5,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5],
  MAX_B_VALUES = [31, 32, 33, 34, 35]
)

In [4]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=164'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [5]:
lst = []
post_str_b1 = None
post_str_b2 = None
post_str_b3 = None
post_str_b4 = None
post_str_blast = None
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_b2)
  cols.append(post_str_b3)
  cols.append(post_str_b4)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_b2 = str_combo[1]
  post_str_b3 = str_combo[2]
  post_str_b4 = str_combo[3]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=7735


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""19/04/2025""","""4191""","""10 06 15 20 21""","""0610152021""","[6, 10, … 21]",6,10,15,20,21,"""06""","""10""","""15""","""20""","""21""",null,null,null,null,null,true,24,"""04_05_05_01""",-3
"""18/04/2025""","""4190""","""21 23 31 05 25""","""0521232531""","[5, 21, … 31]",5,21,23,25,31,"""05""","""21""","""23""","""25""","""31""","""06""","""10""","""15""","""20""","""21""",true,10,"""16_02_02_06""",-10
"""17/04/2025""","""4189""","""33 30 16 19 14""","""1416193033""","[14, 16, … 33]",14,16,19,30,33,"""14""","""16""","""19""","""30""","""33""","""05""","""21""","""23""","""25""","""31""",true,16,"""02_03_11_03""",-23
"""16/04/2025""","""4188""","""21 13 28 29 34""","""1321282934""","[13, 21, … 34]",13,21,28,29,34,"""13""","""21""","""28""","""29""","""34""","""14""","""16""","""19""","""30""","""33""",false,0,"""08_07_01_05""",15
"""15/04/2025""","""4187""","""29 15 23 33 10""","""1015232933""","[10, 15, … 33]",10,15,23,29,33,"""10""","""15""","""23""","""29""","""33""","""13""","""21""","""28""","""29""","""34""",false,0,"""05_08_06_04""",5
"""14/04/2025""","""4186""","""09 27 16 14 04""","""0409141627""","[4, 9, … 27]",4,9,14,16,27,"""04""","""09""","""14""","""16""","""27""","""10""","""15""","""23""","""29""","""33""",true,24,"""05_05_02_11""",15
"""13/04/2025""","""4185""","""26 06 13 09 27""","""0609132627""","[6, 9, … 27]",6,9,13,26,27,"""06""","""09""","""13""","""26""","""27""","""04""","""09""","""14""","""16""","""27""",false,0,"""03_04_13_01""",-29
"""12/04/2025""","""4184""","""23 12 10 16 03""","""0310121623""","[3, 10, … 23]",3,10,12,16,23,"""03""","""10""","""12""","""16""","""23""","""06""","""09""","""13""","""26""","""27""",false,0,"""07_02_04_07""",-6
"""11/04/2025""","""4183""","""09 10 08 01 12""","""0108091012""","[1, 8, … 12]",1,8,9,10,12,"""01""","""08""","""09""","""10""","""12""","""03""","""10""","""12""","""16""","""23""",true,24,"""07_01_01_02""",-5


In [6]:
dfRows.item(0, 'b2')

10

In [7]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str | None = None):
  if test_value == None:
    test_value = dfRows.item(0, col)
    print(f'Setting {test_value=}')

  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()
  mean_percent_75 = mean_percent * 75 / 50
  mean_percent_25 = mean_percent * 25 / 50

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart = chart * hv.HLine(mean_percent_75).opts(color='blue')
  chart = chart * hv.HLine(mean_percent_25).opts(color='blue')
  
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  df_unijump = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'unijump'
    ).agg(pl.count('unijump').alias('count')) \
    .sort("count", descending=True)


  print(f'df_viz FOR {col} = {test_value}')
  display(df_viz)

  display(chart)
  

  print(f'df_10_last FOR {col} = {test_value}')
  last_results = list(set(df_10_last[post_col][0:3]))
  print(f'Last 3 results {last_results=}')
  display(df_10_last)
  
  print(f'df_jumps_map FOR {col} = {test_value}')
  display(df_jumps_map)
  # display(df_unijump)

  print(f'df_unijump FOR {col} = {test_value}')
  display(df_unijump)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list() # Used to discard
  
  return lst_post_str_b, lst_jumps_map, last_results

In [8]:
possible_b1, exclude_jump_maps_b1, last_post_b1 = plot_post_col(col='str_b1', post_col='post_str_b1')

Setting test_value='06'
df_viz FOR str_b1 = 06


post_str_b1,count,percent
str,u32,f64
"""01""",81,14.111498
"""02""",67,11.672474
"""04""",62,10.801394
"""05""",49,8.536585
"""06""",49,8.536585
…,…,…
"""18""",3,0.522648
"""21""",2,0.348432
"""24""",1,0.174216


:Overlay
   .Bars.I    :Bars   [post_str_b1]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b1 = 06
Last 3 results last_results=['04', '05']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""13/04/2025""","""4185""","""26 06 13 09 27""","""0609132627""","[6, 9, … 27]",6,9,13,26,27,"""06""","""09""","""13""","""26""","""27""","""04""","""09""","""14""","""16""","""27""",false,0,"""03_04_13_01""",-29
"""26/03/2025""","""4167""","""34 29 06 24 26""","""0624262934""","[6, 24, … 34]",6,24,26,29,34,"""06""","""24""","""26""","""29""","""34""","""05""","""08""","""27""","""32""","""35""",true,16,"""18_02_03_05""",-16
"""24/03/2025""","""4165""","""32 29 27 21 06""","""0621272932""","[6, 21, … 32]",6,21,27,29,32,"""06""","""21""","""27""","""29""","""32""","""04""","""06""","""22""","""29""","""35""",false,0,"""15_06_02_03""",0
"""10/03/2025""","""4151""","""29 06 26 15 19""","""0615192629""","[6, 15, … 29]",6,15,19,26,29,"""06""","""15""","""19""","""26""","""29""","""02""","""11""","""12""","""16""","""32""",true,16,"""09_04_07_03""",-15
"""05/03/2025""","""4146""","""22 18 16 06 20""","""0616182022""","[6, 16, … 22]",6,16,18,20,22,"""06""","""16""","""18""","""20""","""22""","""13""","""17""","""21""","""30""","""35""",true,28,"""10_02_02_02""",-8
"""06/02/2025""","""4119""","""11 14 25 20 06""","""0611142025""","[6, 11, … 25]",6,11,14,20,25,"""06""","""11""","""14""","""20""","""25""","""08""","""17""","""23""","""28""","""35""",true,16,"""05_03_06_05""",-9
"""04/02/2025""","""4117""","""34 35 06 23 13""","""0613233435""","[6, 13, … 35]",6,13,23,34,35,"""06""","""13""","""23""","""34""","""35""","""14""","""19""","""28""","""31""","""34""",false,0,"""07_10_11_01""",-9
"""30/01/2025""","""4112""","""23 06 22 33 34""","""0622233334""","[6, 22, … 34]",6,22,23,33,34,"""06""","""22""","""23""","""33""","""34""","""04""","""05""","""19""","""28""","""32""",true,16,"""16_01_10_01""",-42
"""25/01/2025""","""4107""","""06 07 25 29 31""","""0607252931""","[6, 7, … 31]",6,7,25,29,31,"""06""","""07""","""25""","""29""","""31""","""03""","""09""","""30""","""32""","""35""",false,0,"""01_18_04_02""",43


df_jumps_map FOR str_b1 = 06


jumps_map,count
str,u32
"""01_03_01_17""",2
"""02_08_06_07""",2
"""13_06_02_06""",2
"""18_01_02_05""",2
"""12_10_04_01""",2
…,…
"""16_07_01_02""",1
"""01_08_12_08""",1
"""13_06_07_02""",1


df_unijump FOR str_b1 = 06


unijump,count
i64,u32
5,15
3,15
0,15
-9,15
7,14
…,…
54,1
58,1
-48,1


In [9]:
possible_b2, exclude_jump_maps_b2, last_post_b2 = plot_post_col(col='str_b2', post_col='post_str_b2')

Setting test_value='10'
df_viz FOR str_b2 = 10


post_str_b2,count,percent
str,u32,f64
"""07""",35,7.12831
"""06""",34,6.924644
"""11""",33,6.720978
"""10""",32,6.517312
"""12""",28,5.702648
…,…,…
"""26""",3,0.610998
"""27""",2,0.407332
"""28""",2,0.407332


:Overlay
   .Bars.I    :Bars   [post_str_b2]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b2 = 10
Last 3 results last_results=['09', '20', '07']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""12/04/2025""","""4184""","""23 12 10 16 03""","""0310121623""","[3, 10, … 23]",3,10,12,16,23,"""03""","""10""","""12""","""16""","""23""","""06""","""09""","""13""","""26""","""27""",false,0,"""07_02_04_07""",-6
"""08/04/2025""","""4180""","""10 20 30 05 16""","""0510162030""","[5, 10, … 30]",5,10,16,20,30,"""05""","""10""","""16""","""20""","""30""","""04""","""20""","""21""","""28""","""29""",false,0,"""05_06_04_10""",11
"""23/01/2025""","""4105""","""26 10 30 24 01""","""0110242630""","[1, 10, … 30]",1,10,24,26,30,"""01""","""10""","""24""","""26""","""30""","""03""","""07""","""15""","""28""","""31""",false,0,"""09_14_02_04""",31
"""04/01/2025""","""4086""","""06 10 29 32 19""","""0610192932""","[6, 10, … 32]",6,10,19,29,32,"""06""","""10""","""19""","""29""","""32""","""07""","""19""","""20""","""23""","""28""",false,0,"""04_09_10_03""",-4
"""20/11/2024""","""4041""","""28 10 07 15 30""","""0710152830""","[7, 10, … 30]",7,10,15,28,30,"""07""","""10""","""15""","""28""","""30""","""12""","""26""","""28""","""32""","""34""",false,0,"""03_05_13_02""",-25
"""19/11/2024""","""4040""","""07 10 21 17 24""","""0710172124""","[7, 10, … 24]",7,10,17,21,24,"""07""","""10""","""17""","""21""","""24""","""07""","""10""","""15""","""28""","""30""",true,16,"""03_07_04_03""",9
"""17/11/2024""","""4038""","""17 25 10 01 14""","""0110141725""","[1, 10, … 25]",1,10,14,17,25,"""01""","""10""","""14""","""17""","""25""","""01""","""07""","""19""","""23""","""24""",true,16,"""09_04_03_08""",2
"""29/09/2024""","""3989""","""03 17 12 10 34""","""0310121734""","[3, 10, … 34]",3,10,12,17,34,"""03""","""10""","""12""","""17""","""34""","""02""","""07""","""09""","""19""","""31""",false,0,"""07_02_05_17""",1
"""04/09/2024""","""3964""","""04 26 29 21 10""","""0410212629""","[4, 10, … 29]",4,10,21,26,29,"""04""","""10""","""21""","""26""","""29""","""03""","""06""","""20""","""31""","""32""",true,16,"""06_11_05_03""",15


df_jumps_map FOR str_b2 = 10


jumps_map,count
str,u32
"""05_01_09_03""",3
"""06_06_03_16""",2
"""05_08_04_13""",2
"""08_19_01_03""",2
"""07_02_04_07""",2
…,…
"""05_02_14_05""",1
"""06_07_12_02""",1
"""09_04_09_02""",1


df_unijump FOR str_b2 = 10


unijump,count
i64,u32
3,11
13,11
11,11
0,10
20,10
…,…
55,1
48,1
-45,1


In [10]:
possible_b3, exclude_jump_maps_b3, last_post_b3 = plot_post_col(col='str_b3', post_col='post_str_b3')

Setting test_value='15'
df_viz FOR str_b3 = 15


post_str_b3,count,percent
str,u32,f64
"""20""",29,6.888361
"""18""",27,6.413302
"""15""",26,6.175772
"""16""",25,5.938242
"""19""",25,5.938242
…,…,…
"""30""",4,0.950119
"""31""",2,0.475059
"""33""",1,0.23753


:Overlay
   .Bars.I    :Bars   [post_str_b3]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b3 = 15
Last 3 results last_results=['21', '18', '22']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""30/03/2025""","""4171""","""35 05 15 29 03""","""0305152935""","[3, 5, … 35]",3,5,15,29,35,"""03""","""05""","""15""","""29""","""35""","""07""","""16""","""22""","""26""","""34""",true,2,"""02_10_14_06""",-8
"""15/03/2025""","""4156""","""16 15 05 27 02""","""0205151627""","[2, 5, … 27]",2,5,15,16,27,"""02""","""05""","""15""","""16""","""27""","""10""","""16""","""21""","""26""","""30""",true,16,"""03_10_01_11""",35
"""12/03/2025""","""4153""","""28 14 35 03 15""","""0314152835""","[3, 14, … 35]",3,14,15,28,35,"""03""","""14""","""15""","""28""","""35""","""07""","""17""","""18""","""19""","""30""",false,0,"""11_01_13_07""",-40
"""09/03/2025""","""4150""","""15 04 16 31 02""","""0204151631""","[2, 4, … 31]",2,4,15,16,31,"""02""","""04""","""15""","""16""","""31""","""06""","""15""","""19""","""26""","""29""",false,0,"""02_11_01_15""",43
"""14/02/2025""","""4127""","""15 09 28 26 14""","""0914152628""","[9, 14, … 28]",9,14,15,26,28,"""09""","""14""","""15""","""26""","""28""","""09""","""21""","""26""","""31""","""32""",true,16,"""05_01_11_02""",-33
"""24/01/2025""","""4106""","""07 03 28 15 31""","""0307152831""","[3, 7, … 31]",3,7,15,28,31,"""03""","""07""","""15""","""28""","""31""","""06""","""07""","""25""","""29""","""31""",false,0,"""04_08_13_03""",-16
"""03/01/2025""","""4085""","""13 04 15 34 18""","""0413151834""","[4, 13, … 34]",4,13,15,18,34,"""04""","""13""","""15""","""18""","""34""","""06""","""10""","""19""","""29""","""32""",true,16,"""09_02_03_16""",4
"""31/12/2024""","""4082""","""01 13 16 24 15""","""0113151624""","[1, 13, … 24]",1,13,15,16,24,"""01""","""13""","""15""","""16""","""24""","""17""","""18""","""29""","""32""","""35""",false,0,"""12_02_01_08""",-1
"""29/12/2024""","""4080""","""15 30 34 01 07""","""0107153034""","[1, 7, … 34]",1,7,15,30,34,"""01""","""07""","""15""","""30""","""34""","""10""","""12""","""17""","""31""","""32""",true,16,"""06_08_15_04""",-23


df_jumps_map FOR str_b3 = 15


jumps_map,count
str,u32
"""12_01_02_09""",2
"""08_03_05_12""",2
"""01_03_06_05""",2
"""01_04_13_07""",2
"""01_03_02_09""",2
…,…
"""04_04_09_08""",1
"""06_06_01_14""",1
"""06_08_16_04""",1


df_unijump FOR str_b3 = 15


unijump,count
i64,u32
-5,14
-14,13
6,12
-3,12
-10,10
…,…
37,1
-45,1
-19,1


In [11]:
possible_b4, exclude_jump_maps_b4, last_post_b4 = plot_post_col(col='str_b4', post_col='post_str_b4')

Setting test_value='20'
df_viz FOR str_b4 = 20


post_str_b4,count,percent
str,u32,f64
"""28""",28,8.408408
"""21""",27,8.108108
"""25""",23,6.906907
"""24""",22,6.606607
"""22""",19,5.705706
…,…,…
"""09""",2,0.600601
"""05""",1,0.3003
"""06""",1,0.3003


:Overlay
   .Bars.I    :Bars   [post_str_b4]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b4 = 20
Last 3 results last_results=['28', '10', '30']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""08/04/2025""","""4180""","""10 20 30 05 16""","""0510162030""","[5, 10, … 30]",5,10,16,20,30,"""05""","""10""","""16""","""20""","""30""","""04""","""20""","""21""","""28""","""29""",false,0,"""05_06_04_10""",11
"""28/03/2025""","""4169""","""20 10 01 09 29""","""0109102029""","[1, 9, … 29]",1,9,10,20,29,"""01""","""09""","""10""","""20""","""29""","""04""","""06""","""09""","""10""","""23""",true,16,"""08_01_10_09""",-26
"""05/03/2025""","""4146""","""22 18 16 06 20""","""0616182022""","[6, 16, … 22]",6,16,18,20,22,"""06""","""16""","""18""","""20""","""22""","""13""","""17""","""21""","""30""","""35""",true,28,"""10_02_02_02""",-8
"""18/02/2025""","""4131""","""26 13 05 20 01""","""0105132026""","[1, 5, … 26]",1,5,13,20,26,"""01""","""05""","""13""","""20""","""26""","""18""","""20""","""21""","""27""","""28""",false,0,"""04_08_07_06""",5
"""06/02/2025""","""4119""","""11 14 25 20 06""","""0611142025""","[6, 11, … 25]",6,11,14,20,25,"""06""","""11""","""14""","""20""","""25""","""08""","""17""","""23""","""28""","""35""",true,16,"""05_03_06_05""",-9
"""16/01/2025""","""4098""","""21 17 20 02 09""","""0209172021""","[2, 9, … 21]",2,9,17,20,21,"""02""","""09""","""17""","""20""","""21""","""02""","""14""","""27""","""29""","""32""",true,16,"""07_08_03_01""",9
"""22/12/2024""","""4073""","""19 20 35 07 05""","""0507192035""","[5, 7, … 35]",5,7,19,20,35,"""05""","""07""","""19""","""20""","""35""","""06""","""09""","""22""","""24""","""32""",false,0,"""02_12_01_15""",46
"""10/12/2024""","""4061""","""05 29 08 20 15""","""0508152029""","[5, 8, … 29]",5,8,15,20,29,"""05""","""08""","""15""","""20""","""29""","""05""","""14""","""20""","""22""","""29""",true,16,"""03_07_05_09""",12
"""12/09/2024""","""3972""","""20 30 02 08 17""","""0208172030""","[2, 8, … 30]",2,8,17,20,30,"""02""","""08""","""17""","""20""","""30""","""05""","""07""","""28""","""30""","""31""",false,0,"""06_09_03_10""",22


df_jumps_map FOR str_b4 = 20


jumps_map,count
str,u32
"""05_01_09_03""",3
"""03_05_07_12""",2
"""08_03_05_12""",2
"""09_01_04_04""",2
"""03_04_01_09""",1
…,…
"""07_07_05_06""",1
"""01_07_09_15""",1
"""03_03_04_03""",1


df_unijump FOR str_b4 = 20


unijump,count
i64,u32
3,13
8,10
-10,10
6,10
18,9
…,…
34,1
37,1
33,1


In [12]:
possible_blast, exclude_jump_maps_blast, last_post_blast = plot_post_col(col='str_b5', post_col='post_str_blast')

Setting test_value='21'
df_viz FOR str_b5 = 21


post_str_blast,count,percent
str,u32,f64
"""35""",20,18.867925
"""31""",13,12.264151
"""32""",9,8.490566
"""34""",7,6.603774
"""27""",7,6.603774
…,…,…
"""21""",3,2.830189
"""23""",2,1.886792
"""20""",2,1.886792


:Overlay
   .Bars.I    :Bars   [post_str_blast]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b5 = 21
Last 3 results last_results=['26', '34', '32']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""16/01/2025""","""4098""","""21 17 20 02 09""","""0209172021""","[2, 9, … 21]",2,9,17,20,21,"""02""","""09""","""17""","""20""","""21""","""02""","""14""","""27""","""29""","""32""",true,16,"""07_08_03_01""",9
"""08/09/2024""","""3968""","""11 06 21 15 13""","""0611131521""","[6, 11, … 21]",6,11,13,15,21,"""06""","""11""","""13""","""15""","""21""","""01""","""14""","""21""","""24""","""26""",true,8,"""05_02_02_06""",1
"""30/08/2024""","""3959""","""07 09 14 21 10""","""0709101421""","[7, 9, … 21]",7,9,10,14,21,"""07""","""09""","""10""","""14""","""21""","""06""","""08""","""25""","""26""","""34""",true,16,"""02_01_04_07""",-4
"""25/01/2024""","""3741""","""10 05 21 04 14""","""0405101421""","[4, 5, … 21]",4,5,10,14,21,"""04""","""05""","""10""","""14""","""21""","""06""","""13""","""17""","""25""","""35""",true,16,"""01_05_04_07""",9
"""12/09/2023""","""3606""","""18 21 12 03 05""","""0305121821""","[3, 5, … 21]",3,5,12,18,21,"""03""","""05""","""12""","""18""","""21""","""13""","""15""","""18""","""31""","""32""",false,0,"""02_07_06_03""",4
"""04/05/2023""","""3475""","""02 07 16 17 21""","""0207161721""","[2, 7, … 21]",2,7,16,17,21,"""02""","""07""","""16""","""17""","""21""","""06""","""08""","""09""","""10""","""33""",true,16,"""05_09_01_04""",23
"""28/04/2023""","""3469""","""11 21 09 15 06""","""0609111521""","[6, 9, … 21]",6,9,11,15,21,"""06""","""09""","""11""","""15""","""21""","""08""","""11""","""12""","""18""","""28""",true,16,"""03_02_04_06""",-3
"""02/04/2023""","""3443""","""01 05 21 03 17""","""0103051721""","[1, 3, … 21]",1,3,5,17,21,"""01""","""03""","""05""","""17""","""21""","""08""","""10""","""15""","""30""","""31""",true,10,"""02_02_12_04""",-28
"""19/02/2023""","""3401""","""02 08 21 16 09""","""0208091621""","[2, 8, … 21]",2,8,9,16,21,"""02""","""08""","""09""","""16""","""21""","""07""","""12""","""25""","""32""","""33""",true,16,"""06_01_07_05""",-19


df_jumps_map FOR str_b5 = 21


jumps_map,count
str,u32
"""01_02_06_09""",1
"""02_06_03_03""",1
"""08_08_03_01""",1
"""02_02_10_05""",1
"""04_01_02_12""",1
…,…
"""04_04_01_04""",1
"""05_05_08_02""",1
"""02_05_11_02""",1


df_unijump FOR str_b5 = 21


unijump,count
i64,u32
-4,7
-14,5
10,5
11,5
-18,5
…,…
32,1
28,1
-16,1


In [13]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [14]:
parquet_file = './parquet_gn_v3.parquet'

# check if parquet_file exists
if not os.path.exists(parquet_file):
  print(f'{parquet_file=} does not exists. creating')
  lst = []
  post_str_b1 = None
  post_str_blast = None
  for row in combos:
    cols = []
    str_row = " ".join([str(r).zfill(2) for r in row])
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)
    cols.append(post_str_b1)
    cols.append(post_str_blast)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)

    post_str_b1 = str_combo[0]
    post_str_blast = str_combo[-1]

  with open('./gn_cahe_v3.json', 'w') as file: 
    json.dump(lst, file)

  df_Orows = pl.DataFrame(
    lst,
    schema=[
      # 'fecha', 'sorteo', 'bolillas',
      'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
      'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
      'post_str_b1', 'post_str_blast',
      'is_rare_combination', 'reason',
      "jumps_map", "unijump"
    ],
    schema_overrides={
      'combo': pl.Array(pl.UInt8, 5),
      'b1': pl.UInt8,
      'b2': pl.UInt8,
      'b3': pl.UInt8,
      'b4': pl.UInt8,
      'b5': pl.UInt8,
      'b6': pl.UInt8,
      'reason': pl.UInt8
    },
    orient='row'
  )

  df_Orows.write_parquet(parquet_file)
  TOTAL_LINES = len(df_Orows)
  print(f'{TOTAL_LINES=} loaded to {parquet_file=}')

  display(df_Orows.head(10))
else:
  df_Orows = pl.read_parquet(parquet_file)

In [15]:
# possible_b1, exclude_jump_maps_b1
df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(possible_b1),
  pl.col("str_b2").is_in(possible_b2),
  pl.col("str_b3").is_in(possible_b3),
  pl.col("str_b4").is_in(possible_b4),
  pl.col("str_b5").is_in(possible_blast),
  
  ~pl.col("str_b1").is_in(last_post_b1),
  ~pl.col("str_b2").is_in(last_post_b2),
  ~pl.col("str_b3").is_in(last_post_b3),
  ~pl.col("str_b4").is_in(last_post_b4),
  ~pl.col("str_b5").is_in(last_post_blast),

  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_b3 + exclude_jump_maps_b4 + exclude_jump_maps_blast),
  pl.col("is_rare_combination") == False
)


display(df_possible)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0104112031""","[1, 4, … 31]",1,4,11,20,31,"""01""","""04""","""11""","""20""","""31""","""01""","""30""",false,0,"""03_07_09_11""",2
"""0104112033""","[1, 4, … 33]",1,4,11,20,33,"""01""","""04""","""11""","""20""","""33""","""01""","""32""",false,0,"""03_07_09_13""",4
"""0104112035""","[1, 4, … 35]",1,4,11,20,35,"""01""","""04""","""11""","""20""","""35""","""01""","""34""",false,0,"""03_07_09_15""",6
"""0104112133""","[1, 4, … 33]",1,4,11,21,33,"""01""","""04""","""11""","""21""","""33""","""01""","""32""",false,0,"""03_07_10_12""",0
"""0104112135""","[1, 4, … 35]",1,4,11,21,35,"""01""","""04""","""11""","""21""","""35""","""01""","""34""",false,0,"""03_07_10_14""",2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1119262735""","[11, 19, … 35]",11,19,26,27,35,"""11""","""19""","""26""","""27""","""35""","""11""","""34""",false,0,"""08_07_01_08""",18
"""1119262933""","[11, 19, … 33]",11,19,26,29,33,"""11""","""19""","""26""","""29""","""33""","""11""","""32""",false,0,"""08_07_03_04""",8
"""1119262935""","[11, 19, … 35]",11,19,26,29,35,"""11""","""19""","""26""","""29""","""35""","""11""","""34""",false,0,"""08_07_03_06""",10


In [16]:
df_possible.sample(n=1)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0812172433""","[8, 12, … 33]",8,12,17,24,33,"""08""","""12""","""17""","""24""","""33""","""08""","""32""",false,0,"""04_05_07_09""",-1
